### Classification techniques. ML in seismic methods.

In [4]:
import glob


In [8]:
import lasio

In [10]:
import numpy as np

In [11]:
# Upload .las files (fies for log data saving)

las_files = []

for p in glob.glob('data/*.las'):
    las_files.append(p)

In [12]:
# Well-log names (headers)
f_cur = 'KOL'
dgk_cur = 'DGR'
aps_cur = 'ASP_EST'
d_cur = 'MD'

In [14]:
# Arrays for writing out data
facies = []
dgk_log = []
aps_log = []
depth_log = []
well_num = []


# Read data from each file and add logs to lists
for item in range(len(las_files)):
    las = lasio.read(las_files[item])
    dgk = las[dgk_cur]
    aps = las[aps_cur]
    depth = las[d_cur]
    ff = las[f_cur]
    
    # Deleting empty samples from logs
    dgk = dgk[~np.isnan(ff)]
    aps = aps[~np.isnan(ff)]
    depth = depth[~np.isnan(ff)]
    ff = ff[~np.isnan(ff)]
    
    depth = depth[~np.isnan(aps)]
    ff = ff[~np.isnan(aps)]
    dgk = dgk[~np.isnan(aps)]
    aps = aps[~np.isnan(aps)]
    
    depth = depth[~np.isnan(dgk)]
    ff = ff[~np.isnan(dgk)]
    aps = aps[~np.isnan(dgk)]
    dgk = dgk[~np.isnan(dgk)]
    
    facies.append(ff)
    dgk_log.append(dgk)
    aps_log.append(aps)
    depth_log.append(depth)
    well_num.append(np.ones(len(ff))*item)

# Merge data into one array
dgk_log = np.concatenate(dgk_log)
aps_log = np.concatenate(aps_log)
depth_log = np.concatenate(depth_log)
facies = np.concatenate(facies)
well_num = np.concatenate(well_num)

In [16]:
# Data Normalisation (we have two well logs providing physical data)
from sklearn import preprocessing

min_max_scaler_1 = preprocessing.MinMaxScaler()
min_max_scaler_2 = preprocessing.MinMaxScaler()

Xn_1 = min_max_scaler_1.fit_transform(dgk_log.reshape(-1, 1))
Xn_2 = min_max_scaler_2.fit_transform(aps_log.reshape(-1, 1))

In [17]:
# Cutting off most informative part
X = np.hstack((Xn_1[(depth_log>1590) & (depth_log<1750)], Xn_2[(depth_log>1590) & (depth_log<1750)]))

y = facies[(depth_log>1590) & (depth_log<1750)].astype(int)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=11)

# примерение различных классификаторов

gnb = GaussianNB() 
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print('GaussianNB')
print(metrics.classification_report(y_test, y_pred))

GaussianNB
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1546
           1       0.74      0.78      0.76       414

    accuracy                           0.90      1960
   macro avg       0.84      0.85      0.85      1960
weighted avg       0.90      0.90      0.90      1960



#### Naive Bayes Classifier

In [161]:
# bayes classifier

# create frequency histogramm (reservoir possibility)
def create_histogramm(positive_values_list, negative_values_list, number_of_columns):
        borders = [0, ]
        positive_probabilities = []
        negative_probabilities = []
        step = 1 / number_of_columns
        
        lower_border = 0
        x = True
        while x: 
            # from 0 to 1 
            # Ones are used in order to avoid Gaussian algos difficulties
            number_of_elements_p_column = 1
            number_of_elements_n_column = 1
            upper_border = lower_border + step
            borders.append(upper_border)
            
            for i in positive_values_list:
                if i > lower_border and i <= upper_border:
                    number_of_elements_p_column += 1
                    
            for j in negative_values_list:
                if j > lower_border and j <= upper_border:
                    number_of_elements_n_column += 1
                    
            lower_border = upper_border
            
            positive_probabilities.append( number_of_elements_p_column )
            negative_probabilities.append( number_of_elements_n_column )
            
            if upper_border >= 1:
                 x = False
        return borders, positive_probabilities, negative_probabilities
    


def bayes_learn(normalised_values, number_of_columns, res):
    histograms_borders = []
    histograms_p_probabilities = []
    histograms_n_probabilities = []
    
    # indexes, where reservoir-layer is located
    positive_indexes = [index for index in range(len(res)) if res[index] == 1]
    # indexes, where non-reservoir-layer is located
    negative_indexes = [index for index in range(len(res)) if res[index] == 0]
    p_values = normalised_values[positive_indexes]
    n_values = normalised_values[negative_indexes]
    borders, p_probabilities, n_probabilities = create_histogramm(p_values, n_values, number_of_columns)
    # returns result for one parameter    
    return borders, p_probabilities, n_probabilities
        
       
        

def bayes_predict(par1, par2, reservour_class_possibity, non_reservoir_class_possibility):
    if par1 == 0: par1 += 10e-9 
    if par2 == 0: par2 += 10e-9 
    # First parameter
    
    positive_first_hist_sum = sum(histograms_p_probabilities1)
    negative_first_hist_sum = sum(histograms_n_probabilities1)
    full_first_sum = positive_first_hist_sum + negative_first_hist_sum
    
    for i in range(len(histograms_borders1)):
        if par1 > histograms_borders1[i] and par1 <= histograms_borders1[i+1]:
            prob_to_others1_p = histograms_p_probabilities1[i] / positive_first_hist_sum
            prob_to_others1_n = histograms_n_probabilities1[i] / negative_first_hist_sum
            full_prob1 = ( histograms_p_probabilities1[i] + histograms_n_probabilities1[i] ) / full_first_sum

    
    

    # Second parameter
    
    positive_second_hist_sum = sum(histograms_p_probabilities2)
    negative_second_hist_sum = sum(histograms_n_probabilities2)
    full_second_sum = positive_second_hist_sum + negative_second_hist_sum
    
    for i in range(len(histograms_borders2)):
        if par2 > histograms_borders2[i] and par2 <= histograms_borders2[i+1]:
            prob_to_others2_p = histograms_p_probabilities2[i] / positive_second_hist_sum
            prob_to_others2_n = histograms_n_probabilities2[i] / negative_second_hist_sum
            full_prob2 = ( histograms_p_probabilities2[i] + histograms_n_probabilities2[i] ) / full_second_sum
    
    prob_resevoir = ( prob_to_others1_p * prob_to_others2_p * reservour_class_possibity ) / (full_prob1 * full_prob2)
    prob_non_resevoir = ( prob_to_others1_n * prob_to_others2_n * non_reservoir_class_possibility ) / (full_prob1 * full_prob2)
    
    
    if prob_resevoir > prob_non_resevoir:
        return 1
    else: 
        return 0
                

In [162]:
whole_samples = len(y)
class_non_reservoir = list(y).count(0)
class_reservoir = whole_samples - class_non_reservoir

reservour_class_possibity = class_reservoir / whole_samples
non_reservoir_class_possibility = class_non_reservoir / whole_samples

print(reservour_class_possibity, non_reservoir_class_possibility)

0.21765756570553713 0.7823424342944628


In [163]:
# Model learning 

# First parameter - GR

histograms_borders1, histograms_p_probabilities1, histograms_n_probabilities1 = bayes_learn(X_train[:, 0], 50, y_train)

# Second parameter - PS

histograms_borders2, histograms_p_probabilities2, histograms_n_probabilities2 = bayes_learn(X_train[:, 1], 50, y_train)

In [164]:
# Prediction

pv = bayes_predict(X_test[2, 0], X_test[2, 1], reservour_class_possibity, non_reservoir_class_possibility)
print(pv)

1


In [165]:
X_test

array([[0.4988203 , 0.3709421 ],
       [0.34625   , 0.        ],
       [0.340625  , 0.4571485 ],
       ...,
       [0.8096874 , 0.        ],
       [0.5914925 , 0.09621976],
       [0.4723231 , 0.4826989 ]])

In [175]:
# Prediction characteristics - Precision and Recall

def precision_test(X_test, y_test):
    class_nr = list(y_test).count(0)
    class_r = list(y_test).count(1)
    
    # Positive
    gotit = 0
    got1 = 0
    for i in range(len(y_test)):
        pv = bayes_predict(X_test[i, 0], X_test[i, 1], reservour_class_possibity, non_reservoir_class_possibility)
        
        if pv == 1:
            got1 += 1
        
        if pv == y_test[i] and y_test[i] == 1:
            gotit += 1
    p_positive = gotit / got1  
    # Negative
    gotit = 0
    got1 = 0
    for i in range(len(y_test)):
        pv = bayes_predict(X_test[i, 0], X_test[i, 1], reservour_class_possibity, non_reservoir_class_possibility)
        
        if pv == 0:
            got1 += 1
        
        if pv == y_test[i] and y_test[i] == 0:
            gotit += 1
    p_negative = gotit / got1   
    
    return p_positive, p_negative
    
    
def recall_test(X_test, y_test):
    class_nr = list(y_test).count(0)
    class_r = list(y_test).count(1)
    
    # Positive
    gotit = 0
    got1 = 0
    for i in range(len(y_test)):
        pv = bayes_predict(X_test[i, 0], X_test[i, 1], reservour_class_possibity, non_reservoir_class_possibility)
        if pv == y_test[i] and y_test[i] == 1:
            gotit += 1
    p_positive = gotit / class_r  
    # Negative
    gotit = 0
    got1 = 0
    for i in range(len(y_test)):
        pv = bayes_predict(X_test[i, 0], X_test[i, 1], reservour_class_possibity, non_reservoir_class_possibility)
        if pv == y_test[i] and y_test[i] == 0:
            gotit += 1
    p_negative = gotit / class_nr  
    
    return p_positive, p_negative
    


In [177]:
presition_positive, presition_negative = precision_test(X_test, y_test)

recall_positive, recall_negative = recall_test(X_test, y_test)

print('Precision (1):', presition_positive)
print('Precision (0):', presition_negative)
print('Recall (1):', recall_positive)
print('Recall (0):', recall_negative)

Precision (1): 0.7912371134020618
Precision (0): 0.9319338422391857
Recall (1): 0.7415458937198067
Recall (0): 0.9476067270375161


...Within next versions of Bayes Naive Classificator we can use Gaussian function for probability approximation instead of histogramms